In [774]:
matches_copa = matches.copy()
decision.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [775]:
matches_copa.columns

Index(['home_team', 'away_team', 'home_score', 'home_xg', 'home_penalty',
       'away_score', 'away_xg', 'away_penalty', 'home_manager', 'home_captain',
       'away_manager', 'away_captain', 'Attendance', 'Venue', 'Officials',
       'Round', 'Date', 'Score', 'Referee', 'Notes', 'Host', 'Year',
       'home_goal', 'away_goal', 'home_goal_long', 'away_goal_long',
       'home_own_goal', 'away_own_goal', 'home_penalty_goal',
       'away_penalty_goal', 'home_penalty_miss_long', 'away_penalty_miss_long',
       'home_penalty_shootout_goal_long', 'away_penalty_shootout_goal_long',
       'home_penalty_shootout_miss_long', 'away_penalty_shootout_miss_long',
       'home_red_card', 'away_red_card', 'home_yellow_red_card',
       'away_yellow_red_card', 'home_yellow_card_long',
       'away_yellow_card_long', 'home_substitute_in_long',
       'away_substitute_in_long'],
      dtype='object')

In [776]:
decision.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral'],
      dtype='object')

In [777]:
# Garante que as datas e anos estão no formato certo
decision['date'] = pd.to_datetime(decision['date'])
decision['Year'] = decision['date'].dt.year
decision.rename(columns={'date': 'match_date'}, inplace=True)

# Copiar decision para cada merge para evitar perda de colunas
decision_home = decision.copy()
decision_away = decision.copy()

# Faz os merges separadamente
matches_copa = matches_copa.merge(decision_home, how='left', on=['home_team', 'Year'], suffixes=('', '_home'))
matches_copa = matches_copa.merge(decision_away, how='left', left_on=['away_team', 'Year'], right_on=['away_team', 'Year'], suffixes=('', '_away'))

# Preenche valores ausentes, se necessário
matches_copa.fillna(0, inplace=True)


C:\Users\VICTUS\AppData\Local\Temp\ipykernel_18476\719889377.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  matches_copa.fillna(0, inplace=True)


In [778]:
matches_copa['Date'] = pd.to_datetime(matches_copa['Date'])
matches_copa['year'] = matches_copa['Date'].dt.year

home_df = matches_copa[[
    'home_team', 'away_team', 'home_score', 'away_score',
    'home_xg', 'away_xg', 'year'
]].copy()
home_df.columns = ['team', 'opponent', 'goals_for', 'goals_against', 'xg_for', 'xg_against', 'year']

away_df = matches_copa[[
    'away_team', 'home_team', 'away_score', 'home_score',
    'away_xg', 'home_xg', 'year'
]].copy()
away_df.columns = ['team', 'opponent', 'goals_for', 'goals_against', 'xg_for', 'xg_against', 'year']

long_df = pd.concat([home_df, away_df], ignore_index=True)

team_stats = long_df.groupby(['team', 'year']).agg({
    'goals_for': 'sum',
    'goals_against': 'sum',
    'xg_for': 'sum',
    'xg_against': 'sum',
    'opponent': 'count' 
}).rename(columns={'opponent': 'matches'}).reset_index()


team_stats = team_stats.merge(
    world_cup[['Year', 'Champion']],
    left_on='year',
    right_on='Year',
    how='left'
)

team_stats['champion'] = (team_stats['team'] == team_stats['Champion']).astype(int)

team_stats.drop(columns=['Champion', 'Year'], inplace=True)

team_stats.head()

,team,year,goals_for,goals_against,xg_for,xg_against,matches,champion
0,Algeria,1982,87,129,0.0,0.0,66,0
1,Algeria,1986,32,208,0.0,0.0,128,0
2,Algeria,2010,0,108,0.0,0.0,188,0
3,Algeria,2014,238,398,0.0,0.0,214,0
4,Angola,2006,11,59,0.0,0.0,147,0


In [779]:
teams_home = matches_copa[['home_team', 'Year']].rename(columns={'home_team': 'Team'})
teams_away = matches_copa[['away_team', 'Year']].rename(columns={'away_team': 'Team'})

teams = pd.concat([teams_home, teams_away], ignore_index=True).drop_duplicates()
teams

,Team,Year
0,Argentina,2022
60,Croatia,2022
141,France,2022
292,Morocco,2022
347,England,2022
...,...,...
78054,United States,1934
78152,Bolivia,1930
78156,Belgium,1930
78169,Mexico,1930


In [780]:
champions = world_cup[['Champion', 'Year']].copy()
champions['Champion_Status'] = True 
champions = champions.rename(columns={'Champion': 'Team'})
champions

,Team,Year,Champion_Status
0,Argentina,2022,True
1,France,2018,True
2,Germany,2014,True
3,Spain,2010,True
4,Italy,2006,True
5,Brazil,2002,True
6,France,1998,True
7,Brazil,1994,True
8,West Germany,1990,True
9,Argentina,1986,True


In [781]:
teams = teams.merge(champions, how='left', on=['Team', 'Year'])
#teams['Champion_Status'] = teams['Champion_Status_x'].fillna(teams['Champion_Status_y'])
#teams.drop(columns=['Champion_Status_x', 'Champion_Status_y'], inplace=True)
teams['Champion_Status'] = teams['Champion_Status'].fillna(False)
teams

C:\Users\VICTUS\AppData\Local\Temp\ipykernel_18476\330077445.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  teams['Champion_Status'] = teams['Champion_Status'].fillna(False)


,Team,Year,Champion_Status
0,Argentina,2022,True
1,Croatia,2022,False
2,France,2022,False
3,Morocco,2022,False
4,England,2022,False
...,...,...,...
484,United States,1934,False
485,Bolivia,1930,False
486,Belgium,1930,False
487,Mexico,1930,False


In [782]:
stats_home = matches_copa.groupby(['home_team', 'Year']).agg({
    'home_score': 'sum',  # Gols feitos em casa
    'away_score': 'sum',  # Gols sofridos em casa
    'home_xg': 'mean',  # Média de Expected Goals em casa
    'home_red_card': 'sum',  # Cartões vermelhos
    'home_yellow_card_long': 'sum'  # Cartões amarelos
}).reset_index()

stats_away = matches_copa.groupby(['away_team', 'Year']).agg({
    'away_score': 'sum',  # Gols feitos fora de casa
    'home_score': 'sum',  # Gols sofridos fora de casa
    'away_xg': 'mean',  # Média de Expected Goals fora de casa
    'away_red_card': 'sum',  # Cartões vermelhos
    'away_yellow_card_long': 'sum'  # Cartões amarelos
}).reset_index()

stats_home.columns = ['Team', 'Year', 'Goals_For_Home', 'Goals_Against_Home', 'XG_Home', 'Red_Cards_Home', 'Yellow_Cards_Home']
stats_away.columns = ['Team', 'Year', 'Goals_For_Away', 'Goals_Against_Away', 'XG_Away', 'Red_Cards_Away', 'Yellow_Cards_Away']

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
# Estatísticas dos jogos como mandante (home team)
stats_home = matches_copa.groupby(['home_team', 'Year']).agg({
    'home_score': 'sum',  # Gols marcados em casa
    'away_score': 'sum',  # Gols sofridos em casa
    'home_xg': 'mean',  # Média de Expected Goals em casa
    'home_red_card': 'sum',  # Cartões vermelhos
    'home_yellow_card_long': 'sum'  # Cartões amarelos
}).reset_index()

stats_home.rename(columns={'home_team': 'Team', 'home_score': 'Goals_For_Home', 'away_score': 'Goals_Against_Home',
                           'home_xg': 'XG_Home', 'home_red_card': 'Red_Cards_Home', 'home_yellow_card_long': 'Yellow_Cards_Home'}, inplace=True)

# Estatísticas dos jogos como visitante (away team)
stats_away = matches_copa.groupby(['away_team', 'Year']).agg({
    'away_score': 'sum',  # Gols marcados fora de casa
    'home_score': 'sum',  # Gols sofridos fora de casa
    'away_xg': 'mean',  # Média de Expected Goals fora de casa
    'away_red_card': 'sum',  # Cartões vermelhos
    'away_yellow_card_long': 'sum'  # Cartões amarelos
}).reset_index()
stats_home.rename(columns={'home_team': 'Team', 'home_score': 'Goals_For_Home', 'away_score': 'Goals_Against_Home',
                           'home_xg': 'XG_Home', 'home_red_card': 'Red_Cards_Home', 'home_yellow_card_long': 'Yellow_Cards_Home'}, inplace=True)

stats_away.rename(columns={'away_team': 'Team', 'away_score': 'Goals_For_Away', 'home_score': 'Goals_Against_Away',
                           'away_xg': 'XG_Away', 'away_red_card': 'Red_Cards_Away', 'away_yellow_card_long': 'Yellow_Cards_Away'}, inplace=True)

In [ ]:
stats_home

,Team,Year,Goals_For_Home,Goals_Against_Home,XG_Home,Red_Cards_Home,Yellow_Cards_Home
0,Algeria,1982,3,4,NaN,0,['67&rsquor;|0:2|Faouzi Mansouri']
1,Algeria,1986,1,4,NaN,0,['33&rsquor;|0:1|Rabah Madjer']['36&rsquor;|0:...
2,Algeria,2010,0,1,NaN,0,"['59&rsquor;|0:0|Abdelkader Ghezzal', '90+5&rs..."
3,Algeria,2014,1,1,NaN,0,"['39&rsquor;|0:1|Djamel Mesbah', '87&rsquor;|1..."
4,Angola,2006,0,1,NaN,0,"['28&rsquor;|0:1|Jamba', '45+3&rsquor;|0:1|Loc..."
...,...,...,...,...,...,...,...
425,Yugoslavia,1962,9,1,NaN,Vladica Popović · 71,0
426,Yugoslavia,1974,9,2,NaN,0,"['36&rsquor;|0:0|Ivan Buljan', '61&rsquor;|0:1..."
427,Yugoslavia,1982,0,0,NaN,0,0
428,Yugoslavia,1990,5,1,NaN,Refik Šabanadžović · 31,['24&rsquor;|0:0|Refik Šabanadžović']['20&rsqu...


In [ ]:
# 1. Juntar as estatísticas de casa e fora
teams = pd.merge(stats_home, stats_away, on=['Team', 'Year'], how='outer')

# 2. Converter as colunas numéricas para tipo numérico (garantir que podem ser somadas)
numeric_cols = ['Goals_For_Home', 'Goals_Against_Home', 'Goals_For_Away', 'Goals_Against_Away',
                'XG_Home', 'XG_Away', 'Red_Cards_Home', 'Red_Cards_Away',
                'Yellow_Cards_Home', 'Yellow_Cards_Away']

for col in numeric_cols:
    teams[col] = pd.to_numeric(teams[col], errors='coerce')

# 3. Calcular totais combinados
teams['Goals_For'] = teams['Goals_For_Home'] + teams['Goals_For_Away']
teams['Goals_Against'] = teams['Goals_Against_Home'] + teams['Goals_Against_Away']
teams['XG'] = (teams['XG_Home'] + teams['XG_Away']) / 2
teams['Red_Cards'] = teams['Red_Cards_Home'] + teams['Red_Cards_Away']
teams['Yellow_Cards'] = teams['Yellow_Cards_Home'] + teams['Yellow_Cards_Away']

# 4. Remover colunas intermediárias, se desejar
teams.drop(columns=numeric_cols, inplace=True)
teams.fillna(0, inplace=True)
# 5. Exibir resultado
teams.head()


,Team,Year,Goals_For,Goals_Against,XG,Red_Cards,Yellow_Cards
0,Algeria,1982,5.0,5.0,0.0,0.0,0.0
1,Algeria,1986,1.0,5.0,0.0,0.0,0.0
2,Algeria,2010,0.0,2.0,0.0,0.0,0.0
3,Algeria,2014,7.0,7.0,0.0,0.0,0.0
4,Angola,2006,1.0,2.0,0.0,0.0,0.0


In [ ]:
# Evitar divisão por zero e garantir que XG seja numérico
teams['Goals_Efficiency'] = teams['Goals_For'] / teams['XG']
teams['Goals_Efficiency'] = teams['Goals_Efficiency'].replace([float('inf'), float('-inf')], 0)  # Substituir valores inválidos por 0
teams.fillna({'Goals_Efficiency': 0}, inplace=True)  # Substituir NaN por 0

In [ ]:
# Estatísticas de pênaltis como mandante
penalty_home = matches_copa.groupby(['home_team', 'Year']).agg({
    'home_penalty_goal': 'sum',
    'home_penalty_miss_long': 'sum'
}).reset_index()

penalty_home.rename(columns={'home_team': 'Team', 'home_penalty_goal': 'Penalty_Goals_Home', 
                             'home_penalty_miss_long': 'Penalty_Misses_Home'}, inplace=True)

# Estatísticas de pênaltis como visitante
penalty_away = matches_copa.groupby(['away_team', 'Year']).agg({
    'away_penalty_goal': 'sum',
    'away_penalty_miss_long': 'sum'
}).reset_index()

penalty_away.rename(columns={'away_team': 'Team', 'away_penalty_goal': 'Penalty_Goals_Away', 
                             'away_penalty_miss_long': 'Penalty_Misses_Away'}, inplace=True)

In [ ]:
teams = teams.merge(penalty_home, on=['Team', 'Year'], how='left')
teams = teams.merge(penalty_away, on=['Team', 'Year'], how='left')

# Preencher valores NaN com 0 para evitar erros
teams.fillna(0, inplace=True)

In [ ]:
# Garantir que as colunas de pênaltis sejam numéricas
columns_to_convert = ['Penalty_Goals_Home', 'Penalty_Goals_Away', 'Penalty_Misses_Home', 'Penalty_Misses_Away']

for col in columns_to_convert:
    teams[col] = pd.to_numeric(teams[col], errors='coerce')  # Converte para número e substitui erros com NaN

# Agora podemos somar sem erro
teams['Penalty_Goals'] = teams['Penalty_Goals_Home'] + teams['Penalty_Goals_Away']
teams['Penalty_Misses'] = teams['Penalty_Misses_Home'] + teams['Penalty_Misses_Away']

# Calcular eficiência nos pênaltis
teams['Penalty_Efficiency'] = teams['Penalty_Goals'] / (teams['Penalty_Goals'] + teams['Penalty_Misses'])
teams['Penalty_Efficiency'] = teams['Penalty_Efficiency'].replace([float('inf'), float('-inf')], 0)  # Substituir valores inválidos por 0
teams.fillna({'Penalty_Efficiency': 0}, inplace=True)  # Garantir que não existam NaN

In [ ]:
teams.drop(columns=['Penalty_Goals_Home', 'Penalty_Goals_Away', 'Penalty_Misses_Home', 'Penalty_Misses_Away'], inplace=True)

In [ ]:
champions_df = world_cup[['Champion', 'Year']].copy()
champions_df['Champion_Status'] = True  

teams = teams.merge(champions_df, how='left', left_on=['Team', 'Year'], right_on=['Champion', 'Year'])

teams['Champion_Status'] = teams['Champion_Status'].fillna(False)

teams.drop(columns=['Champion'], inplace=True)

C:\Users\VICTUS\AppData\Local\Temp\ipykernel_18476\2160151589.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  teams['Champion_Status'] = teams['Champion_Status'].fillna(False)


In [ ]:
initial_elo = 1500
base_K = 40  
elo_ratings = {}

# Ordenar os dados por ano para garantir consistência na atualização do Elo
matches_copa = matches_copa.sort_values(by='year')

# Loop por cada ano de Copa
for year in matches_copa['year'].unique():
    copa_ano = matches_copa[matches_copa['year'] == year]

    for index, row in copa_ano.iterrows():
        team_home = row['home_team']
        team_away = row['away_team']
        score_home = row['home_score']
        score_away = row['away_score']

        # Inicializa Elo
        for team in [team_home, team_away]:
            if team not in elo_ratings:
                elo_ratings[team] = initial_elo

        # Expectativa
        expected_home = 1 / (1 + 10**((elo_ratings[team_away] - elo_ratings[team_home]) / 400))
        expected_away = 1 - expected_home

        # Resultado real
        if score_home > score_away:
            result_home, result_away = 1, 0
        elif score_home < score_away:
            result_home, result_away = 0, 1
        else:
            result_home = result_away = 0.5

        # Diferença de gols influencia o K
        goal_diff = abs(score_home - score_away)
        multiplier = np.log(goal_diff + 1) + 1 

        # Atualização sem ajuste por força histórica
        K = base_K * multiplier
        elo_ratings[team_home] += K * (result_home - expected_home)
        elo_ratings[team_away] += K * (result_away - expected_away)

    # Após o fim da Copa, aplicar bônus para o campeão
    champion = world_cup.loc[world_cup['Year'] == year, 'Champion'].values[0]
    if champion in elo_ratings:
        elo_ratings[champion] += 100  # bônus de campeão

# Transformar em DataFrame
elo_df = pd.DataFrame(list(elo_ratings.items()), columns=['Team', 'Elo_Rating_Adjusted'])

# Mesclar com teams
teams = teams.merge(elo_df, on='Team', how='left')


In [ ]:
# Transformar Elo Ratings em DataFrame
elo_df = pd.DataFrame(list(elo_ratings.items()), columns=['Team', 'Elo_Rating_Adjusted'])

# Mesclar com o DataFrame 'teams'
teams = teams.merge(elo_df, on='Team', how='left')



In [ ]:
teams['Elo_Rating_Adjusted'] = teams['Elo_Rating_Adjusted_x']

teams = teams.drop(columns=['Elo_Rating_Adjusted_x', 'Elo_Rating_Adjusted_y'])

print(teams[['Team', 'Year', 'Elo_Rating_Adjusted']].head())


      Team  Year  Elo_Rating_Adjusted
0  Algeria  1982          1439.748479
1  Algeria  1986          1439.748479
2  Algeria  2010          1439.748479
3  Algeria  2014          1439.748479
4   Angola  2006          1475.499947


In [ ]:
decision.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [ ]:
teams

,Team,Year,Goals_For,Goals_Against,XG,Red_Cards,Yellow_Cards,Goals_Efficiency,Penalty_Goals,Penalty_Misses,Penalty_Efficiency,Champion_Status,Elo_Rating_Adjusted
0,Algeria,1982,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1439.748479
1,Algeria,1986,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1439.748479
2,Algeria,2010,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1439.748479
3,Algeria,2014,7.0,7.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,False,1439.748479
4,Angola,2006,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1475.499947
...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,Yugoslavia,1962,10.0,7.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,False,1628.631700
485,Yugoslavia,1974,12.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1628.631700
486,Yugoslavia,1982,2.0,2.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,False,1628.631700
487,Yugoslavia,1990,8.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1628.631700
